In [ ]:
#input dataset information by developers
project_dir <- "./Data/Spatial/visium/kidney/mouse_NTN/"
processed_dir <- "./Data/Spatial/visium/kidney/mouse_NTN/processed/ultimate_processed/"
pipeline_dir <- "./software/project_repos/visium_pipeline"

suppressWarnings(suppressMessages(library(Seurat)))
suppressWarnings(suppressMessages(library(dplyr)))
suppressWarnings(suppressMessages(library(tidyverse)))
suppressMessages(source(paste0(pipeline_dir, "/R/utils.R")))

library(ggpubr)
library(patchwork)
library(ggthemes)

# reading data

In [ ]:
sc_obj <- readRDS("./Data/scRNA/kidney/mouse_NTN/processed/annotated_v2.rds")
sc_obj

integrated_obj <- readRDS(file.path(processed_dir, "integrated/rds/integrated_annotated_2.rds"))
message("Integrated object read")

An object of class Seurat 
29159 features across 19431 samples within 2 assays 
Active assay: SCT (14578 features, 3000 variable features)
 3 layers present: counts, data, scale.data
 1 other assay present: RNA
 3 dimensional reductions calculated: pca, harmony, umap

Integrated object read



In [3]:
MyPalette <- c("#CBD588","#89C5DA", "#DA5724", "#74D944", "#CE50CA", "#3F4921", "#C0717C",  "#5F7FC7", 
"#673770", "#D3D93E", "#38333E", "#508578", "#D7C1B1", "#689030", "#AD6F3B", "#CD9BCD", 
"#D14285", "#6DDE88", "#652926", "#7FDCC0", "#C84248", "#8569D5", "#5E738F", "#D1A33D", 
"#8A7C64", "#599861")

In [4]:
sc_obj@meta.data %>% colnames()
Idents(sc_obj) <- "annotation"
integrated_obj$Treatment <- integrated_obj$diagnosis %>% factor(levels= c("Healthy", "vehicle", "Sema", "Enalapril")) 

[1] "orig.ident"         "nCount_RNA"         "nFeature_RNA"      
 [4] "alias_id"           "Treatment"          "log10GenesPerUMI"  
 [7] "percent_mito"       "nCount_SCT"         "nFeature_SCT"      
[10] "SCT_snn_res.0.1"    "SCT_snn_res.0.2"    "SCT_snn_res.0.3"   
[13] "SCT_snn_res.0.4"    "SCT_snn_res.0.5"    "SCT_snn_res.0.6"   
[16] "SCT_snn_res.0.7"    "SCT_snn_res.0.8"    "seurat_clusters"   
[19] "clustering_harmony" "first_annotation"   "second_annotation" 
[22] "third_annotation"   "annotation"

# SPATIAL 

In [5]:
cell_types <- read.csv(paste0(project_dir, "/processed/deconvolution/reference_signatures/reference_cell_type_signature.csv"), row.names = 1) %>% 
colnames()
cell_types

[1] "Adipocytes" "CNT"        "DCT"        "EC"         "FIB"       
 [6] "IC"         "IMM"        "PC"         "POD"        "PTS1"      
[11] "PTS2"       "PTS3"       "ThickL"     "ThinL"      "Urothelium"
[16] "VSCM"

In [ ]:
#sample 
options(repr.plot.width=10, repr.plot.height=5)
colour_genus <- integrated_obj$Region2 %>% unique()
palette <- MyPalette[1:length(colour_genus)]
names(palette) <- colour_genus

p <- integrated_obj@meta.data %>%
  dplyr::count(alias_id, Treatment, Region2) %>%
  group_by(alias_id) %>%
  mutate(fraction = n/sum(n)) %>%
  ggplot(aes(alias_id, fraction, fill = Region2)) +
  geom_col() +
  scale_fill_manual(values = palette) +
  cowplot::theme_cowplot() +
  scale_x_discrete(guide = guide_axis(angle = 45)) +
  labs(x = "", y = "", fill = "Region") +
  facet_wrap(~ Treatment, scales = "free_x", nrow = 1)

ggsave("./kidney_mouse_ntn/reports/plots/cell_type_composition/all/ST_region_sample_treatment.png", p, width = 10, height = 5, bg = "white")

In [ ]:
#treatment 
options(repr.plot.width=10, repr.plot.height=5)
colour_genus <- integrated_obj$Region2 %>% unique()
palette <- MyPalette[1:length(colour_genus)]
names(palette) <- colour_genus

p <- integrated_obj@meta.data %>%
  dplyr::count(Treatment, Region2) %>%
  group_by(Treatment) %>%
  mutate(fraction = n/sum(n)) %>%
  ggplot(aes(Treatment, fraction, fill = Region2)) +
  geom_col() +
  scale_fill_manual(values = palette) +
  cowplot::theme_cowplot() +
  scale_x_discrete(guide = guide_axis(angle = 45)) +
  labs(x = "", y = "", fill = "Region") +
  facet_wrap(~ Treatment, scales = "free_x", nrow = 1) +
  theme(axis.text.x=element_blank(), axis.ticks.x=element_blank(), axis.line=element_blank())

ggsave(".//reports/plots/cell_type_composition/all/ST_region_treatment.png", p, width = 10, height = 5, bg = "white")

In [ ]:
#treatment sample
options(repr.plot.width=10, repr.plot.height=5)
palette <- MyPalette[1:length(cell_types)]
names(palette) <- cell_types

p <- integrated_obj@meta.data %>% 
    select(c(alias_id,Treatment,cell_types)) %>%
    gather(cell_type, count, -c(Treatment, alias_id)) %>%
    group_by(alias_id, cell_type, Treatment) %>% 
    summarise(n = sum(count, na.rm = TRUE)) %>% 
    group_by(alias_id) %>% 
    mutate(fraction = n/sum(n)) %>%
    ggplot(aes(alias_id, fraction, fill = cell_type)) +
    geom_col() +
    scale_fill_manual(values = palette) +
    cowplot::theme_cowplot() +
    scale_x_discrete(guide = guide_axis(angle = 45)) +
    labs(x = "", y = "", fill = "Region") +
    facet_wrap(~ Treatment, scales = "free_x", nrow = 1)

ggsave(".//reports/plots/cell_type_composition/all/ST_cell_type_sample_treatment.png", p, width = 10, height = 5, bg = "white")

Warning message:
“Using an external vector in selections was deprecated in tidyselect 1.1.0.
ℹ Please use `all_of()` or `any_of()` instead.
  # Was:
  data %>% select(cell_types)

  # Now:
  data %>% select(all_of(cell_types))

See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>.”


`summarise()` has grouped output by 'alias_id', 'cell_type'. You can override using the `.groups` argument.


In [ ]:
#treatment cell types
options(repr.plot.width=10, repr.plot.height=5)
palette <- MyPalette[1:length(cell_types)]
names(palette) <- cell_types

p <- integrated_obj@meta.data %>% 
    select(c(Treatment,cell_types)) %>%
    gather(cell_type, count, -Treatment) %>% 
    group_by(Treatment, cell_type) %>% 
    summarise(n = sum(count, na.rm = TRUE)) %>% 
    group_by(Treatment) %>% 
    mutate(fraction = n/sum(n)) %>%
    ggplot(aes(Treatment, fraction, fill = cell_type)) +
    geom_col() +
    scale_fill_manual(values = palette) +
    cowplot::theme_cowplot() +
    scale_x_discrete(guide = guide_axis(angle = 45)) +
    labs(x = "", y = "", fill = "Region") +
    facet_wrap(~ Treatment, scales = "free_x", nrow = 1) +
    theme(axis.text.x=element_blank(), axis.ticks.x=element_blank(), axis.line=element_blank())

ggsave(".//reports/plots/cell_type_composition/all/ST_region_treatment.png", p, width = 10, height = 5, bg = "white")

`summarise()` has grouped output by 'Treatment'. You can override using the `.groups` argument.


# Single cell

In [ ]:
#SINGLE CELL
options(repr.plot.width=10, repr.plot.height=5)

palette <- MyPalette[1:length(cell_types)]
names(palette) <- cell_types

p <- sc_obj@meta.data %>%
  dplyr::count(alias_id, Treatment, annotation) %>%
  group_by(alias_id) %>%
  mutate(fraction = n/sum(n),
         alias_id = paste0(alias_id)) %>%
  ggplot(aes(alias_id, fraction, fill = annotation)) +
  geom_col() +
  scale_fill_manual(values = palette) +
  cowplot::theme_cowplot() +
  scale_x_discrete(guide = guide_axis(angle = 45)) +
  labs(x = "", y = "", fill = "Cell type") +
  facet_wrap(~ Treatment, scales = "free_x", nrow = 1)

ggsave(".//reports/plots/cell_type_composition/all/sc_cell_type_sample_treatment.png", p, width = 10, height = 5, bg = "white")

In [ ]:
#SINGLE CELL
options(repr.plot.width=10, repr.plot.height=5)

colour_genus <- sc_obj$annotation %>% unique()
palette <- MyPalette[1:length(colour_genus)]
names(palette) <- colour_genus

p <- sc_obj@meta.data %>%
  dplyr::count(Treatment, annotation) %>%
  group_by(Treatment) %>%
  mutate(fraction = n/sum(n),) %>%
  ggplot(aes(Treatment, fraction, fill = annotation)) +
  geom_col() +
  scale_fill_manual(values = palette) +
  cowplot::theme_cowplot() +
  scale_x_discrete(guide = guide_axis(angle = 45)) +
  labs(x = "", y = "", fill = "Cell type") +
  facet_wrap(~ Treatment, scales = "free_x", nrow = 1) +
  theme(axis.text.x=element_blank(), axis.ticks.x=element_blank(), axis.line=element_blank())

p <- ggsave(".//reports/plots/cell_type_composition/all/sc_cell_type_treatment.png", p, width = 10, height = 5, bg = "white")